In [1]:
import os
import json
import cv2
import numpy as np
import logging
import onnxruntime as ort
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from typing import List
from scipy.special import expit
from Modules import LineDetection
from Utils import create_dir, get_file_name, patch_image, unpatch_image, resize_image

In [3]:
line_model_config = "Models/LineModels/line_model_config.json"
ocr_model_config = "Models/OCRModels/LhasaKanjur/model_config.json"
line_inference = LineDetection(line_model_config)

In [4]:
image_path = "Data\W30125"
images = glob(f"{image_path}/*.jpg")
print(f"Image: {len(images)}")

out_path = os.path.join(image_path, "predictions")
create_dir(out_path)

Image: 7


In [5]:
for _, image_path in tqdm(enumerate(images), total=len(images)):
    image_name = get_file_name(image_path)
    image = cv2.imread(image_path)
    prediction = line_inference.predict(image)

    cv2.addWeighted(prediction, 0.4, image, 1-0.4, 0, image)
    out_prediction = f"{out_path}/{image_name}_prediction.jpg"
    cv2.imwrite(out_prediction, image)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:47<00:00,  6.77s/it]
